In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from math import sqrt

In [2]:
df = pd.read_csv(
    'df_final.csv', 
    parse_dates=['time']
)

In [3]:
df.columns

Index(['time', 'generation biomass', 'generation fossil brown coal/lignite',
       'generation fossil gas', 'generation fossil hard coal',
       'generation fossil oil', 'generation hydro pumped storage consumption',
       'generation hydro run-of-river and poundage',
       'generation hydro water reservoir', 'generation nuclear',
       'generation other', 'generation other renewable', 'generation solar',
       'generation waste', 'generation wind onshore', 'total load actual',
       'price day ahead', 'price actual', 'temp_Barcelona',
       'temp_min_Barcelona', 'temp_max_Barcelona', 'pressure_Barcelona',
       'humidity_Barcelona', 'wind_speed_Barcelona', 'wind_deg_Barcelona',
       'rain_1h_Barcelona', 'rain_3h_Barcelona', 'snow_3h_Barcelona',
       'clouds_all_Barcelona', 'weather_id_Barcelona',
       'weather_main_Barcelona', 'weather_description_Barcelona',
       'weather_icon_Barcelona', 'temp_Bilbao', 'temp_min_Bilbao',
       'temp_max_Bilbao', 'pressure_Bilbao', 

In [4]:
columnas_energía = ['generation biomass', 'generation fossil brown coal/lignite',
       'generation fossil gas', 'generation fossil hard coal',
       'generation fossil oil', 'generation hydro pumped storage consumption',
       'generation hydro run-of-river and poundage',
       'generation hydro water reservoir', 'generation nuclear',
       'generation other', 'generation other renewable', 'generation solar',
       'generation waste', 'generation wind onshore']

columnas_lluvia = ["rain_1h_Barcelona","rain_1h_Bilbao","rain_1h_Madrid","rain_1h_Seville","rain_1h_Valencia"]

columna_precio = ["price actual"]

In [5]:
df['time']

0       2014-12-31 23:00:00+00:00
1       2015-01-01 00:00:00+00:00
2       2015-01-01 01:00:00+00:00
3       2015-01-01 02:00:00+00:00
4       2015-01-01 03:00:00+00:00
                   ...           
35059   2018-12-31 18:00:00+00:00
35060   2018-12-31 19:00:00+00:00
35061   2018-12-31 20:00:00+00:00
35062   2018-12-31 21:00:00+00:00
35063   2018-12-31 22:00:00+00:00
Name: time, Length: 35064, dtype: datetime64[ns, UTC]

In [6]:
df_gr_energia_mean = df.groupby([df['time'].dt.year.rename('year'), df['time'].dt.month.rename('month')])[columnas_energía].mean().reset_index()
df_gr_energia_sum = df.groupby([df['time'].dt.year.rename('year'), df['time'].dt.month.rename('month')])[columnas_energía].sum().reset_index()
df_gr_precio = df.groupby([df['time'].dt.year.rename('year'), df['time'].dt.month.rename('month')])[columna_precio].mean().reset_index()
df_gr_lluvia = df.groupby([df['time'].dt.year.rename('year'), df['time'].dt.month.rename('month')])[columnas_lluvia].sum().reset_index()

In [7]:
df_gr_energia_mean['renovable'] = df_gr_energia_mean['generation hydro pumped storage consumption'] + df_gr_energia_mean['generation hydro run-of-river and poundage'] + df_gr_energia_mean['generation hydro water reservoir'] + df_gr_energia_mean['generation solar'] + df_gr_energia_mean['generation wind onshore'] + df_gr_energia_mean['generation other renewable']
df_gr_energia_mean['no_renovable'] = df_gr_energia_mean['generation biomass'] + df_gr_energia_mean['generation fossil brown coal/lignite'] + df_gr_energia_mean['generation fossil gas'] + df_gr_energia_mean['generation fossil hard coal'] + df_gr_energia_mean['generation fossil oil'] + df_gr_energia_mean['generation nuclear'] + df_gr_energia_mean['generation other'] + df_gr_energia_mean['generation waste']

In [8]:
df_gr_energia_sum['renovable'] = df_gr_energia_sum['generation hydro pumped storage consumption'] + df_gr_energia_sum['generation hydro run-of-river and poundage'] + df_gr_energia_sum['generation hydro water reservoir'] + df_gr_energia_sum['generation solar'] + df_gr_energia_sum['generation wind onshore'] + df_gr_energia_sum['generation other renewable']
df_gr_energia_sum['no_renovable'] = df_gr_energia_sum['generation biomass'] + df_gr_energia_sum['generation fossil brown coal/lignite'] + df_gr_energia_sum['generation fossil gas'] + df_gr_energia_sum['generation fossil hard coal'] + df_gr_energia_sum['generation fossil oil'] + df_gr_energia_sum['generation nuclear'] + df_gr_energia_sum['generation other'] + df_gr_energia_sum['generation waste']

In [9]:
def construct_path(year, month):
    return f'{year:d}/{month:02d}'

In [10]:
df_gr_energia_mean['date'] = df_gr_energia_mean.apply(lambda x: construct_path(int(x['year']),int(x["month"])), axis=1)
df_gr_energia_sum['date'] = df_gr_energia_sum.apply(lambda x: construct_path(int(x['year']),int(x["month"])), axis=1)
df_gr_precio['date'] = df_gr_precio.apply(lambda x: construct_path(int(x['year']),int(x["month"])), axis=1)
df_gr_lluvia['date'] = df_gr_lluvia.apply(lambda x: construct_path(int(x['year']),int(x["month"])), axis=1)

In [11]:
df_gr_lluvia['total'] = df_gr_lluvia.apply(lambda x: x['rain_1h_Barcelona']+x['rain_1h_Bilbao']+x['rain_1h_Madrid']+x['rain_1h_Seville']+x['rain_1h_Valencia'], axis=1)

In [12]:
df_gr_energia_mean.set_index("date")
df_gr_energia_sum.set_index("date")
df_gr_precio.set_index("date")
df_gr_lluvia.set_index("date")

,year,month,rain_1h_Barcelona,rain_1h_Bilbao,rain_1h_Madrid,rain_1h_Seville,rain_1h_Valencia,total
date,,,,,,,,
2014/12,2014,12,0.0,0.0,0.0,0.0,0.00,0.00
2015/01,2015,1,64.8,382.5,25.8,40.2,10.20,523.50
2015/02,2015,2,218.1,487.8,29.4,24.6,25.20,785.10
2015/03,2015,3,261.0,397.8,134.1,152.7,360.30,1305.90
2015/04,2015,4,45.0,141.3,84.3,111.9,47.40,429.90
2015/05,2015,5,33.3,378.9,6.9,6.9,15.00,441.00
2015/06,2015,6,163.5,144.0,22.5,9.6,49.80,389.40
2015/07,2015,7,77.1,26.1,0.9,3.6,8.40,116.10
2015/08,2015,8,148.5,24.6,3.6,1.8,39.00,217.50


In [14]:
df_gr_energia_mean_y = df.groupby([df['time'].dt.year.rename('year')])[columnas_energía].mean().reset_index()
df_gr_energia_sum_y = df.groupby([df['time'].dt.year.rename('year')])[columnas_energía].sum().reset_index()
df_gr_precio_y = df.groupby([df['time'].dt.year.rename('year')])[columna_precio].mean().reset_index()
df_gr_lluvia_y = df.groupby([df['time'].dt.year.rename('year')])[columnas_lluvia].sum().reset_index()

In [15]:
df_gr_energia_mean_y['renovable'] = df_gr_energia_mean_y['generation hydro pumped storage consumption'] + df_gr_energia_mean_y['generation hydro run-of-river and poundage'] + df_gr_energia_mean_y['generation hydro water reservoir'] + df_gr_energia_mean_y['generation solar'] + df_gr_energia_mean_y['generation wind onshore'] + df_gr_energia_mean_y['generation other renewable']
df_gr_energia_mean_y['no_renovable'] = df_gr_energia_mean_y['generation biomass'] + df_gr_energia_mean_y['generation fossil brown coal/lignite'] + df_gr_energia_mean_y['generation fossil gas'] + df_gr_energia_mean_y['generation fossil hard coal'] + df_gr_energia_mean_y['generation fossil oil'] + df_gr_energia_mean_y['generation nuclear'] + df_gr_energia_mean_y['generation other'] + df_gr_energia_mean_y['generation waste']

In [16]:
df_gr_lluvia_y['total'] = df_gr_lluvia_y.apply(lambda x: x['rain_1h_Barcelona']+x['rain_1h_Bilbao']+x['rain_1h_Madrid']+x['rain_1h_Seville']+x['rain_1h_Valencia'], axis=1)

In [17]:
df_gr_energia_mean_y.set_index("year")
df_gr_energia_sum_y.set_index("year")
df_gr_precio_y.set_index("year")
df_gr_lluvia_y.set_index("year")

,rain_1h_Barcelona,rain_1h_Bilbao,rain_1h_Madrid,rain_1h_Seville,rain_1h_Valencia,total
year,,,,,,
2014,0.0,0.0,0.0,0.0,0.00,0.00
2015,1290.3,2197.8,375.9,446.1,664.20,4974.30
2016,1769.1,822.6,403.2,353.1,305.54,3653.54
2017,329.4,566.4,214.8,238.8,103.20,1452.60
2018,415.2,591.3,524.4,404.4,135.30,2070.60


In [18]:
df_gr_lluvia_y

,year,rain_1h_Barcelona,rain_1h_Bilbao,rain_1h_Madrid,rain_1h_Seville,rain_1h_Valencia,total
0,2014,0.0,0.0,0.0,0.0,0.00,0.00
1,2015,1290.3,2197.8,375.9,446.1,664.20,4974.30
2,2016,1769.1,822.6,403.2,353.1,305.54,3653.54
3,2017,329.4,566.4,214.8,238.8,103.20,1452.60
4,2018,415.2,591.3,524.4,404.4,135.30,2070.60


In [19]:
df_gr_lluvia_y.to_csv("df_gr_lluvia_y.csv")

In [24]:
df_gr_lluvia.to_csv("df_gr_lluvia.csv")

In [20]:
df_gr_energia_mean_y.to_csv("df_gr_energia_mean_y.csv")

In [21]:
df_gr_precio.to_csv("df_gr_precio.csv")

In [22]:
df_gr_precio_y.to_csv("df_gr_precio_y.csv")

In [23]:
df_gr_precio_y

,year,price actual
0,2014,65.410000
1,2015,61.359949
2,2016,47.438307
3,2017,59.321965
4,2018,63.444265
